In [50]:
import pandas as pd
import numpy as np
import re
import math
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import (Embedding, Conv1D, GlobalMaxPooling1D, Dense, GRU, Flatten, 
                                     Dropout, GlobalAveragePooling2D, Conv2D, MaxPooling2D, BatchNormalization, LeakyReLU)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.applications import EfficientNetB1

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer, snowball
from nltk.corpus import stopwords

import pickle



In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [60]:
###Les fichiers csv proviennent du challenge Rakuten
### Une copie sur ce google drive : https://drive.google.com/drive/folders/1PltQt7eFWu5lkGf4jRdqqRIZaykBlta8?usp=drive_link

In [56]:
X_train = pd.read_csv('X_train_update.csv', index_col=0)

In [5]:
y_train = pd.read_csv("Y_train.csv", index_col=0)

In [6]:
from unidecode import unidecode

def remove_accents(text):
    return unidecode(text) if pd.notnull(text) else text

X_train['designation'] = X_train['designation'].apply(remove_accents)
X_train['description'] = X_train['description'].apply(remove_accents)

In [7]:
# Convertir les majuscules en minuscules
X_train['designation'] = X_train['designation'].str.lower()
X_train['description'] = X_train['description'].str.lower()

# Supprimer les nombres 
X_train['designation'] = X_train['designation'].str.replace('[0-9]', '', regex=True)
X_train['designation'] = X_train['designation'].str.replace('[^\w\s]', '', regex=True)

# et les caractères spéciaux/ponctuation
X_train['description'] = X_train['description'].str.replace('[0-9]', '', regex=True)
X_train['description'] = X_train['description'].str.replace('[^\w\s]', '', regex=True)

In [8]:
# Preprocessing stopword francais/anglais/allemand

stop_words = set(stopwords.words('french'))

# On va créer une liste comportant tous les stop words français, anglais et allemand
ang = set(stopwords.words('english'))
ang = list(ang)
ger = set(stopwords.words('german'))
ger = list(ger)

# Création d'une fonction pour ajouter des mots stop words
def ajout(list):
    for i in list:
        stop_words.add(i)
        
ajout(ang)
ajout(ger)

In [9]:
# Assuming stop_words is already defined
# stop_words = set(stopwords.words('english'))

def process_text(text):
    # Tokenize and remove single characters
    tokenizer = RegexpTokenizer(r'\b[a-z]{2,}\b')
    tokens = tokenizer.tokenize(text.lower())

    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(token) for token in tokens]

    # Remove stopwords
    cleaned = [word for word in lemmatized if word not in stop_words]

    return ' '.join(cleaned)

# Apply the function to the DataFrame
for col in ['designation', 'description']:
    X_train[col] = X_train[col].fillna('').apply(process_text)


In [10]:
# Fill missing values with a placeholder
X_train.fillna('nodat', inplace=True)

# Concatenate the 'designation' and 'description' columns
# This avoids the need to use word_tokenize at this stage

X_train['combined'] = X_train['designation'] + ' ' + X_train['description']

X_train_final = X_train['combined']



In [11]:
y_train_final =y_train.prdtypecode

In [12]:
### encodage identique de y pour les 2 models text et image
import joblib
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train_final)

joblib.dump(le, 'label_encoder.joblib')


# ###One-hot

y_train_onehot=to_categorical(y_train_encoded, num_classes=27)



In [51]:
###sauver le labelencoder

with open("label_encoder.pickle", "wb") as handle:
    pickle.dump(le, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [13]:
# Création de nos ensemble d'apprentissage et test

X_trainn, X_testt, y_trainn, y_testt = train_test_split(X_train_final, y_train_onehot, test_size=0.2, random_state = 42)

In [16]:

train_size = int(X_trainn.shape[0])
train_posts = X_trainn[:train_size]
train_tags = y_trainn[:train_size]

train_size = int(X_testt.shape[0])
test_posts = X_testt[:train_size]
test_tags = y_testt[:train_size]

# Create a tokenizer and fit it on the training posts
tokenizer = Tokenizer(num_words=None, char_level=False)
tokenizer.fit_on_texts(train_posts)

# Convert texts to sequences for both training and testing sets
x_train = tokenizer.texts_to_sequences(train_posts)
x_test = tokenizer.texts_to_sequences(test_posts)

vocab_size = len(tokenizer.word_index) + 1

# Padding sequences
max_len = 100
x_train_text = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_len)
x_test_text = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_len)


In [54]:
##sauver tokenizer
with open("tokenizer.pickle", "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [17]:
# Dataset de notre jeu de données

text_train_set =tf.data.Dataset.from_tensor_slices((x_train_text, y_trainn))
text_test_set = tf.data.Dataset.from_tensor_slices((x_test_text, y_testt))

In [18]:
# Ajouter la fonction load_image dans le pipeline des opérations. Séparer le résultat en lot de taille 32.
text_train_set = text_train_set.map(lambda text, y: [text, y]).batch(32).repeat(-1)
text_test_set = text_test_set.map(lambda text, y: [text, y]).batch(32).repeat(-1)

In [19]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Flatten, Dropout


input_layer = Input(shape=(100,))  
embedding_layer = Embedding(input_dim=vocab_size, output_dim=100, input_length=100)(input_layer)
conv_layer = Conv1D(filters=128, kernel_size=5, activation='relu')(embedding_layer)
global_max_pooling_layer = GlobalMaxPooling1D()(conv_layer)

output_layer = Dense(1024, activation='relu')(global_max_pooling_layer)
model_te = Model(inputs=input_layer, outputs=output_layer)

model_te.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding (Embedding)       (None, 100, 100)          17862800  
                                                                 
 conv1d (Conv1D)             (None, 96, 128)           64128     
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 1024)              132096    
                                                                 
Total params: 18,059,024
Trainable params: 18,059,024
Non-trainable params: 0
_________________________________________________

In [21]:
##model image

In [46]:
import glob
import pandas as pd

##images d'origine telechargez sur le challenge Rakuten 500*500 et avec bord blanc non viré

liste_train = glob.glob('../../Projet_Rakuten/images/images/image_train/*.jpg')
liste_test = glob.glob('../../Projet_Rakuten/images/images/image_test/*.jpg')

In [23]:
###pour enlever le probleme de \ /
liste_train_clean=[]
for i in liste_train :
    liste_train_clean.append(i.replace("\\","/"))

In [24]:
path_train=r'./X_train_update.csv'
X_train=pd.read_csv(path_train, index_col=0)

In [25]:
### extraction des noms d images pour pouvoir faire le merge des 2 df
import re
r=re.compile(r"image_[0-9]+")
r2=re.compile(r"[0-9]+")
liste_image=[]             

for path in liste_train :
    image=r.findall(path)[0]
    liste_image.append(r2.findall(image)[0])

In [31]:
df=pd.DataFrame({'imageid':liste_image, 'path':liste_train_clean})

In [33]:
### split 80/20
X_train_image, X_test_image, y_train, y_test = train_test_split(df.path, y_train_onehot, test_size=0.2, random_state=42)

In [34]:
np.array_equal(y_test, y_testt)

True

In [35]:


@tf.function
def load_image(filepath, resize=(200, 200)):
    im = tf.io.read_file(filepath)
    im = tf.image.decode_png(im, channels=3)
    return tf.image.resize(im, resize, method='nearest')

# def load_image_train(filepath, resize=(200, 200)):
#     im = tf.io.read_file(filepath)
#     im = tf.image.decode_png(im, channels=3)
#     return tf.image.resize(im, resize, method='nearest')

# Chargement des données
dataset_train = tf.data.Dataset.from_tensor_slices((X_train_image, y_train))
dataset_train = dataset_train.map(lambda x, y : [load_image(x), y], num_parallel_calls=-1).batch(32).repeat(-1)

dataset_test =tf.data.Dataset.from_tensor_slices((X_test_image, y_test))
dataset_test = dataset_test.map(lambda x, y : [load_image(x), y], num_parallel_calls=-1).batch(32).repeat(-1)

In [36]:
#### import de EfficientNetB1 et freeze du backbone

from tensorflow.keras.applications import EfficientNetB1

####### Changer le input_shape avec la bonne resolution

efficientNet = EfficientNetB1(include_top=False, input_shape=(200, 200, 3))

for layer in efficientNet.layers:
    layer.trainable = False

    
efficientNet.summary()

Model: "efficientnetb1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 200, 200, 3)  0           ['input_2[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 200, 200, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 rescaling_1 (Rescaling)        (None, 200, 200, 3)  0           ['normalization[0][0

 block2a_expand_conv (Conv2D)   (None, 100, 100, 96  1536        ['block1b_add[0][0]']            
                                )                                                                 
                                                                                                  
 block2a_expand_bn (BatchNormal  (None, 100, 100, 96  384        ['block2a_expand_conv[0][0]']    
 ization)                       )                                                                 
                                                                                                  
 block2a_expand_activation (Act  (None, 100, 100, 96  0          ['block2a_expand_bn[0][0]']      
 ivation)                       )                                                                 
                                                                                                  
 block2a_dwconv_pad (ZeroPaddin  (None, 101, 101, 96  0          ['block2a_expand_activation[0][0]
 g2D)     

 ivation)                                                                                         
                                                                                                  
 block2c_dwconv (DepthwiseConv2  (None, 50, 50, 144)  1296       ['block2c_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block2c_bn (BatchNormalization  (None, 50, 50, 144)  576        ['block2c_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block2c_activation (Activation  (None, 50, 50, 144)  0          ['block2c_bn[0][0]']             
 )                                                                                                
          

                                                                                                  
 block3b_activation (Activation  (None, 25, 25, 240)  0          ['block3b_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block3b_se_squeeze (GlobalAver  (None, 240)         0           ['block3b_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block3b_se_reshape (Reshape)   (None, 1, 1, 240)    0           ['block3b_se_squeeze[0][0]']     
                                                                                                  
 block3b_se_reduce (Conv2D)     (None, 1, 1, 10)     2410        ['block3b_se_reshape[0][0]']     
          

 )                                                                                                
                                                                                                  
 block4a_se_squeeze (GlobalAver  (None, 240)         0           ['block4a_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block4a_se_reshape (Reshape)   (None, 1, 1, 240)    0           ['block4a_se_squeeze[0][0]']     
                                                                                                  
 block4a_se_reduce (Conv2D)     (None, 1, 1, 10)     2410        ['block4a_se_reshape[0][0]']     
                                                                                                  
 block4a_se_expand (Conv2D)     (None, 1, 1, 240)    2640        ['block4a_se_reduce[0][0]']      
          

...

                                                                                                  
 block5a_project_bn (BatchNorma  (None, 13, 13, 112)  448        ['block5a_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block5b_expand_conv (Conv2D)   (None, 13, 13, 672)  75264       ['block5a_project_bn[0][0]']     
                                                                                                  
 block5b_expand_bn (BatchNormal  (None, 13, 13, 672)  2688       ['block5b_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block5b_expand_activation (Act  (None, 13, 13, 672)  0          ['block5b_expand_bn[0][0]']      
 ivation) 

                                                                                                  
 block5d_expand_bn (BatchNormal  (None, 13, 13, 672)  2688       ['block5d_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block5d_expand_activation (Act  (None, 13, 13, 672)  0          ['block5d_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block5d_dwconv (DepthwiseConv2  (None, 13, 13, 672)  16800      ['block5d_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block5d_b

 block6b_dwconv (DepthwiseConv2  (None, 7, 7, 1152)  28800       ['block6b_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block6b_bn (BatchNormalization  (None, 7, 7, 1152)  4608        ['block6b_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block6b_activation (Activation  (None, 7, 7, 1152)  0           ['block6b_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block6b_se_squeeze (GlobalAver  (None, 1152)        0           ['block6b_activation[0][0]']     
 agePoolin

                                                                                                  
 block6d_activation (Activation  (None, 7, 7, 1152)  0           ['block6d_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block6d_se_squeeze (GlobalAver  (None, 1152)        0           ['block6d_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block6d_se_reshape (Reshape)   (None, 1, 1, 1152)   0           ['block6d_se_squeeze[0][0]']     
                                                                                                  
 block6d_se_reduce (Conv2D)     (None, 1, 1, 48)     55344       ['block6d_se_reshape[0][0]']     
          

 agePooling2D)                                                                                    
                                                                                                  
 block7a_se_reshape (Reshape)   (None, 1, 1, 1152)   0           ['block7a_se_squeeze[0][0]']     
                                                                                                  
 block7a_se_reduce (Conv2D)     (None, 1, 1, 48)     55344       ['block7a_se_reshape[0][0]']     
                                                                                                  
 block7a_se_expand (Conv2D)     (None, 1, 1, 1152)   56448       ['block7a_se_reduce[0][0]']      
                                                                                                  
 block7a_se_excite (Multiply)   (None, 7, 7, 1152)   0           ['block7a_activation[0][0]',     
                                                                  'block7a_se_expand[0][0]']      
          

In [37]:

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Conv2D, MaxPooling2D, BatchNormalization, LeakyReLU, Flatten
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras import layers


# Modèle
inputs_image = Input(shape=(200, 200, 3))
x = efficientNet(inputs_image)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(27, activation='softmax')(x)

model_im = Model(inputs=inputs_image, outputs=outputs)
model_im.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 efficientnetb1 (Functional)  (None, 7, 7, 1280)       6575239   
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense_1 (Dense)             (None, 1024)              1311744   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_2 (Dense)             (None, 27)                27675     
                                                           

In [38]:
### on defreeze les 4 dernieres layers
for layer in efficientNet.layers[-4:]:
    layer.trainable = True

In [40]:
# Définition d'un générateur python
def generator(image_set, treated_text_set):
    iter_image = iter(image_set)
    iter_text_treated = iter(treated_text_set)
    while True:
        X_im, y = next(iter_image)
        X_text_treated, y_text = next(iter_text_treated) 
        yield [X_im, X_text_treated], y_text

In [41]:
# Définition du générateur final.
gen_train = generator(dataset_train, text_train_set)
gen_test = generator(dataset_test, text_test_set)

In [42]:
from tensorflow.keras.layers import Concatenate, Attention
from tensorflow.keras import Model

attention_output = Attention()([x, output_layer])
concatenated = Concatenate()([attention_output, x, output_layer])

concatenated = Dense(1024, activation='relu',name='dense_' + 'concat')(concatenated)
concatenated = Dropout(0.3,name= 'dropout_' + 'concat')(concatenated)
output = layers.Dense(27, activation='softmax')(concatenated)

model = Model([inputs_image, input_layer], output)

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_1 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 efficientnetb1 (Functional)    (None, 7, 7, 1280)   6575239     ['input_3[0][0]']                
                                                                                                  
 embedding (Embedding)          (None, 100, 100)     17862800    ['input_1[0][0]']          

In [43]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [44]:
from tensorflow.keras import callbacks

# Savegarde automatique des poids
checkpoint = callbacks.ModelCheckpoint(filepath='checkpoint', 
                                       monitor='val_loss',
                                       save_best_only=True,
                                       save_weights_only=False,
                                       mode='min',
                                       save_freq='epoch')

# Réduction automatique du taux d'apprentissage
lr_plateau = callbacks.ReduceLROnPlateau(monitor='val_loss',
                                         patience=2,
                                         factor=0.1,
                                         verbose=2,
                                         mode='min')

early_stopping = callbacks.EarlyStopping(monitor='val_loss',
                                         patience=3,
                                         mode='min',
                                         restore_best_weights=True)


In [45]:
import math

train_steps = math.ceil(len(y_train)/32)
validation_steps = math.ceil(len(y_test)/32)

model.fit(
    x=gen_train,
    steps_per_epoch = train_steps,
    validation_data = gen_test,
    validation_steps = validation_steps,
    verbose=1,
    epochs=10,
    callbacks=[lr_plateau, early_stopping]
)

Epoch 1/10
2123/2123 [==============================] - 265s 108ms/step - loss: 1.2927 - accuracy: 0.6136 - val_loss: 0.8202 - val_accuracy: 0.7571 - lr: 0.0010
Epoch 2/10
2123/2123 [==============================] - 126s 60ms/step - loss: 0.5094 - accuracy: 0.8437 - val_loss: 0.8568 - val_accuracy: 0.7706 - lr: 0.0010
Epoch 3/10
2122/2123 [============================>.] - ETA: 0s - loss: 0.2499 - accuracy: 0.9230
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
2123/2123 [==============================] - 119s 56ms/step - loss: 0.2499 - accuracy: 0.9230 - val_loss: 0.9766 - val_accuracy: 0.7747 - lr: 0.0010
Epoch 4/10
2123/2123 [==============================] - 119s 56ms/step - loss: 0.0976 - accuracy: 0.9702 - val_loss: 1.0181 - val_accuracy: 0.7889 - lr: 1.0000e-04


In [47]:

# validation_steps = math.ceil(len(y_test)/32)
# y_pred = model.predict(gen_test, steps=validation_steps, verbose = 1)

In [48]:


def evaluate_model(model, gen_test, validation_steps):
    y_true = []
    y_pred = []

    for _ in tqdm(range(validation_steps), desc="Evaluating", unit="step"):
        (X_im, X_text_treated), y_text = next(gen_test)
        y_true.extend(y_text.numpy())
        y_pred_batch = model.predict([X_im, X_text_treated], verbose=0)
        y_pred.extend(y_pred_batch)

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    y_pred_labels = np.argmax(y_pred, axis=1)
    y_true_labels = np.argmax(y_true, axis=1)

    print(classification_report(y_true_labels, y_pred_labels))

# Utilisez la fonction
gen_test = generator(dataset_test, text_test_set)
evaluate_model(model, gen_test, validation_steps)



Evaluating: 100%|██████████| 531/531 [00:55<00:00,  9.58step/s]

              precision    recall  f1-score   support

           0       0.33      0.44      0.38       612
           1       0.58      0.63      0.60       521
           2       0.80      0.63      0.71       357
           3       0.98      0.76      0.86       161
           4       0.76      0.73      0.74       539
           5       0.88      0.92      0.90       786
           6       0.65      0.28      0.39       146
           7       0.61      0.57      0.59       961
           8       0.55      0.44      0.49       424
           9       0.83      0.86      0.85       974
          10       0.93      0.76      0.83       169
          11       0.73      0.74      0.74       507
          12       0.84      0.59      0.70       672
          13       0.89      0.69      0.78      1013
          14       0.88      0.91      0.89       841
          15       0.69      0.85      0.76       137
          16       0.73      0.74      0.74      1029
          17       0.40    

In [49]:
model.save('bimodal.h5', include_optimizer=False)
